#  heat budget using SASSIE ECCO model 

*Produced by Ali Siddiqui, following the notebook previously prepared by Marie Zahn <br>
Updated May 30, 2025*

***

This notebook calculates the heat budget for one day using instantaneous snapshots and daily means of data variables.

Here we follow procedures outlined in the [ECCO tutorial](https://ecco-v4-python-tutorial.readthedocs.io/ECCO_v4_Heat_budget_closure.html#) and this [MITgcm reference](http://mitgcm.org/download/daily_snapshot/MITgcm/doc/Heat_Salt_Budget_MITgcm.pdf) by Chakraborty & Jean-Michel Campin. In summary:

For the SASSIE LLC1080 model, we do not apply a scaling factor due to the linear free surface approximation. Accordingly, the heat budget is formulated as
\begin{equation}
\underbrace{\frac{\partial(\theta)}{\partial t}}_{G^{\theta}_\textrm{total}} = \underbrace{-\nabla_{z^{*}} \cdot(\theta\,\mathbf{v}_{res}) - \frac{\partial(\theta\,w_{res})}{\partial z^{*}}}_{G^{\theta}_\textrm{advection}}\underbrace{-({\nabla\cdot\mathbf{F}_\textrm{diff}^{\theta}})}_{G^{\theta}_\textrm{diffusion}} + \underbrace{{F}_\textrm{forc}^{\theta}}_{G^{\theta}_\textrm{forcing}}
\end{equation}

***

In [1]:
## Initalize Python libraries
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import s3fs
import fsspec
from glob import glob

import matplotlib.pyplot as plt
from datetime import datetime
import os
from pathlib import Path
import s3fs
import boto3
import numpy as np
import pandas as pd
from pathlib import Path

import boto3
import s3fs
import fsspec
import xarray as xr
import zarr

import dask
from dask.distributed import Client, LocalCluster

In [2]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:39841")
client

<Client: 'tcp://127.0.0.1:39841' processes=20 threads=80, memory=616.49 GiB>

In [3]:
import warnings
warnings.filterwarnings("ignore")

# load sassie fields

## Open ZARR store

In [4]:
def get_aws_credentials(profile_name='sassie_read'):
    session = boto3.Session(profile_name=profile_name)
    credentials = session.get_credentials()
    return credentials

In [5]:
# load sassie profile credentials
aws_credentials = get_aws_credentials(profile_name='sassie_read')
# initialize s3 filesystem
s3_options = dict(anon=False, key=aws_credentials.access_key, secret=aws_credentials.secret_key)

In [6]:
# function to open zarr store with a provided s3 bucket path
def open_zarr_store(s3_path, s3_options):

    # initalize s3 file system
    fs = fsspec.filesystem("s3", asynchronous=True, **s3_options)

    # define location of zarr store and open
    store = zarr.storage.FsspecStore(fs, path=s3_path)
    zarr_store = xr.open_dataset(store, engine='zarr',consolidated=False)
    
    return zarr_store

In [7]:
salt_path   = 'podaac-dev-sassie/ECCO_model/N1/V1R1/HH/ZARR/SALT_AVG_DAILY.ZARR/'
grid_path   = 'podaac-dev-sassie/ECCO_model/N1/V1R1/HH/GRID/GRID_GEOMETRY_SASSIE_HH_V1R1_NATIVE_LLC1080.ZARR/'
theta_path  = 'podaac-dev-sassie/ECCO_model/N1/V1R1/HH/ZARR/THETA_AVG_DAILY.ZARR/'

ADVx_TH_path = 'podaac-dev-sassie/ECCO_model/N1/V1R1/HH/ZARR/ADVx_TH_AVG_DAILY.ZARR/'
ADVy_TH_path = 'podaac-dev-sassie/ECCO_model/N1/V1R1/HH/ZARR/ADVy_TH_AVG_DAILY.ZARR/'
ADVr_TH_path = 'podaac-dev-sassie/ECCO_model/N1/V1R1/HH/ZARR/ADVr_TH_AVG_DAILY.ZARR/'
DFrI_TH_path = 'podaac-dev-sassie/ECCO_model/N1/V1R1/HH/ZARR/DFrI_TH_AVG_DAILY.ZARR/'
TFLUX_path = 'podaac-dev-sassie/ECCO_model/N1/V1R1/HH/ZARR/TFLUX_AVG_DAILY.ZARR/'
# KPPg_TH_path = 'podaac-dev-sassie/ECCO_model/N1/V1R1/HH/ZARR/KPPdiffT_AVG_DAILY.ZARR/'
oceQsw_path = 'podaac-dev-sassie/ECCO_model/N1/V1R1/HH/ZARR/oceQsw_AVG_DAILY.ZARR/'
# WTHMASS_ds = 'podaac-dev-sassie/ECCO_model/N1/V1R1/HH/ZARR/KPPdiffT_AVG_DAILY.ZARR/'


In [8]:
# open SALT zarr store
SALT_zarr = open_zarr_store(salt_path, s3_options)
SALT_zarr

<xarray.Dataset> Size: 4TB
Dimensions:    (time: 2581, k: 90, j: 1080, i: 1800, k_l: 90, k_u: 90,
                k_p1: 91, nv: 2)
Coordinates: (12/14)
    Z          (k) float32 360B ...
    Zl         (k_l) float32 360B ...
    YC         (j, i) float32 8MB ...
    XC         (j, i) float32 8MB ...
  * j          (j) int32 4kB 0 1 2 3 4 5 6 ... 1074 1075 1076 1077 1078 1079
    Zu         (k_u) float32 360B ...
    ...         ...
  * k          (k) int32 360B 0 1 2 3 4 5 6 7 8 9 ... 81 82 83 84 85 86 87 88 89
  * k_l        (k_l) int32 360B 0 1 2 3 4 5 6 7 8 ... 81 82 83 84 85 86 87 88 89
  * k_p1       (k_p1) int32 364B 0 1 2 3 4 5 6 7 8 ... 83 84 85 86 87 88 89 90
  * k_u        (k_u) int32 360B 0 1 2 3 4 5 6 7 8 ... 81 82 83 84 85 86 87 88 89
    time_bnds  (time, nv) datetime64[ns] 41kB ...
  * time       (time) datetime64[ns] 21kB 2014-01-15T12:00:00 ... 2021-02-07T...
Dimensions without coordinates: nv
Data variables:
    SALT       (time, k, j, i) float64 4TB ...
Attributes: (12/55)
    Conventions:                     CF-1.8, ACDD-1.3
    acknowledgement:                 This research was carried out by the Jet...
    author:                          Mike Wood, Marie Zahn, and Ian Fenty
    cdm_data_type:                   Grid
    comment:                         SASSIE llc1080 V1R1 fields are consolida...
    creator_email:                   ecco-group@mit.edu
    ...                              ...
    standard_name_vocabulary:        NetCDF Climate and Forecast (CF) Metadat...
    summary:                         This dataset provides data variable and ...
    time_coverage_end:               2021-02-08T00:00:00
    time_coverage_start:             2014-01-15T00:00:00
    title:                           SASSIE Ocean Model SALT Parameter for th...
    uuid:                            ee3d4cde-1381-11ef-9c99-0604868e061f

### Open geometry file for HH

In [9]:
HH_grid = open_zarr_store(grid_path, s3_options)
HH_grid

<xarray.Dataset> Size: 6GB
Dimensions:     (j: 1080, i: 1800, k: 90, k_p1: 91, nb: 4, j_g: 1080,
                 i_g: 1800, k_l: 90, nv: 2, k_u: 90)
Coordinates: (12/19)
    XC          (j, i) float32 8MB ...
    XC_bnds     (j, i, nb) float32 31MB ...
    YG          (j_g, i_g) float32 8MB ...
    XG          (j_g, i_g) float32 8MB ...
    YC          (j, i) float32 8MB ...
    Z           (k) float32 360B ...
    ...          ...
  * i_g         (i_g) int32 7kB 0 1 2 3 4 5 6 ... 1794 1795 1796 1797 1798 1799
  * j_g         (j_g) int32 4kB 0 1 2 3 4 5 6 ... 1074 1075 1076 1077 1078 1079
  * k_l         (k_l) int32 360B 0 1 2 3 4 5 6 7 8 ... 82 83 84 85 86 87 88 89
  * k_u         (k_u) int32 360B 0 1 2 3 4 5 6 7 8 ... 82 83 84 85 86 87 88 89
  * k_p1        (k_p1) int32 364B 0 1 2 3 4 5 6 7 8 ... 83 84 85 86 87 88 89 90
  * k           (k) int32 360B 0 1 2 3 4 5 6 7 8 ... 81 82 83 84 85 86 87 88 89
Dimensions without coordinates: nb, nv
Data variables: (12/22)
    CS          (j, i) float64 16MB ...
    PHrefC      (k) float64 720B ...
    PHrefF      (k_p1) float64 728B ...
    Depth       (j, i) float64 16MB ...
    SN          (j, i) float64 16MB ...
    drC         (k_p1) float64 728B ...
    ...          ...
    rAc         (j, i) float64 16MB ...
    rAs         (j_g, i) float64 16MB ...
    mask_basin  (k, j, i) int64 1GB ...
    rAz         (j_g, i_g) float64 16MB ...
    rAw         (j, i_g) float64 16MB ...
    maskW       (k, j, i_g) bool 175MB ...
Attributes: (12/58)
    Conventions:                     CF-1.8, ACDD-1.3
    acknowledgement:                 This research was carried out by the Jet...
    author:                          Mike Wood, Marie Zahn, and Ian Fenty
    cdm_data_type:                   Grid
    comment:                         Fields provided on the curvilinear lat-l...
    coordinates_comment:             Note: the global 'coordinates' attribute...
    ...                              ...
    references:                      TBD
    source:                          The SASSIE ocean model simulation was pr...
    standard_name_vocabulary:        NetCDF Climate and Forecast (CF) Metadat...
    summary:                         This dataset provides geometric paramete...
    title:                           SASSIE Ocean Model Geometry Parameters f...
    uuid:                            85e78ca0-e7f6-11ee-ad4d-aa598f4e49fc

### Open `THETA` snapshots

In [10]:
theta_snaps = open_zarr_store(theta_path, s3_options)
theta_snaps

<xarray.Dataset> Size: 4TB
Dimensions:    (time: 2581, k: 90, j: 1080, i: 1800, k_p1: 91, k_l: 90,
                k_u: 90, nv: 2)
Coordinates: (12/14)
    XC         (j, i) float32 8MB ...
    YC         (j, i) float32 8MB ...
    Zp1        (k_p1) float32 364B ...
  * i          (i) int32 7kB 0 1 2 3 4 5 6 ... 1794 1795 1796 1797 1798 1799
    Zl         (k_l) float32 360B ...
    Z          (k) float32 360B ...
    ...         ...
  * k_l        (k_l) int32 360B 0 1 2 3 4 5 6 7 8 ... 81 82 83 84 85 86 87 88 89
  * k_p1       (k_p1) int32 364B 0 1 2 3 4 5 6 7 8 ... 83 84 85 86 87 88 89 90
  * time       (time) datetime64[ns] 21kB 2014-01-15T12:00:00 ... 2021-02-07T...
  * j          (j) int32 4kB 0 1 2 3 4 5 6 ... 1074 1075 1076 1077 1078 1079
    time_bnds  (time, nv) datetime64[ns] 41kB ...
  * k_u        (k_u) int32 360B 0 1 2 3 4 5 6 7 8 ... 81 82 83 84 85 86 87 88 89
Dimensions without coordinates: nv
Data variables:
    THETA      (time, k, j, i) float64 4TB ...
Attributes: (12/55)
    Conventions:                     CF-1.8, ACDD-1.3
    acknowledgement:                 This research was carried out by the Jet...
    author:                          Mike Wood, Marie Zahn, and Ian Fenty
    cdm_data_type:                   Grid
    comment:                         SASSIE llc1080 V1R1 fields are consolida...
    creator_email:                   ecco-group@mit.edu
    ...                              ...
    standard_name_vocabulary:        NetCDF Climate and Forecast (CF) Metadat...
    summary:                         This dataset provides data variable and ...
    time_coverage_end:               2021-02-08T00:00:00
    time_coverage_start:             2014-01-15T00:00:00
    title:                           SASSIE Ocean Model THETA Parameter for t...
    uuid:                            92f116d8-13ba-11ef-9cf8-0604868e061f

In [11]:
# see what dates we have for snapshots
theta_snaps.time.values

array(['2014-01-15T12:00:00.000000000', '2014-01-16T12:00:00.000000000',
       '2014-01-17T12:00:00.000000000', ...,
       '2021-02-05T12:00:00.000000000', '2021-02-06T12:00:00.000000000',
       '2021-02-07T12:00:00.000000000'], dtype='datetime64[ns]')

### Resample to monthly means

In [12]:
# Select first two snaps
# theta_daily_snaps = theta_snaps.isel(time=slice(0,2))

In [13]:
#We use the wonderful power of pandas to chose  the first day of each month for the snapshot terms
theta_snaps = theta_snaps.sel(time=theta_snaps['time'].dt.is_month_start)
theta_snaps

<xarray.Dataset> Size: 119GB
Dimensions:    (time: 85, k: 90, j: 1080, i: 1800, k_p1: 91, k_l: 90, k_u: 90,
                nv: 2)
Coordinates: (12/14)
    XC         (j, i) float32 8MB ...
    YC         (j, i) float32 8MB ...
    Zp1        (k_p1) float32 364B ...
  * i          (i) int32 7kB 0 1 2 3 4 5 6 ... 1794 1795 1796 1797 1798 1799
    Zl         (k_l) float32 360B ...
    Z          (k) float32 360B ...
    ...         ...
  * k_l        (k_l) int32 360B 0 1 2 3 4 5 6 7 8 ... 81 82 83 84 85 86 87 88 89
  * k_p1       (k_p1) int32 364B 0 1 2 3 4 5 6 7 8 ... 83 84 85 86 87 88 89 90
  * time       (time) datetime64[ns] 680B 2014-02-01T12:00:00 ... 2021-02-01T...
  * j          (j) int32 4kB 0 1 2 3 4 5 6 ... 1074 1075 1076 1077 1078 1079
    time_bnds  (time, nv) datetime64[ns] 1kB ...
  * k_u        (k_u) int32 360B 0 1 2 3 4 5 6 7 8 ... 81 82 83 84 85 86 87 88 89
Dimensions without coordinates: nv
Data variables:
    THETA      (time, k, j, i) float64 119GB ...
Attributes: (12/55)
    Conventions:                     CF-1.8, ACDD-1.3
    acknowledgement:                 This research was carried out by the Jet...
    author:                          Mike Wood, Marie Zahn, and Ian Fenty
    cdm_data_type:                   Grid
    comment:                         SASSIE llc1080 V1R1 fields are consolida...
    creator_email:                   ecco-group@mit.edu
    ...                              ...
    standard_name_vocabulary:        NetCDF Climate and Forecast (CF) Metadat...
    summary:                         This dataset provides data variable and ...
    time_coverage_end:               2021-02-08T00:00:00
    time_coverage_start:             2014-01-15T00:00:00
    title:                           SASSIE Ocean Model THETA Parameter for t...
    uuid:                            92f116d8-13ba-11ef-9cf8-0604868e061f

In [14]:
theta_snaps.nbytes*1e-9

118.988368448

In [15]:
# check dates
theta_snaps.time.values

array(['2014-02-01T12:00:00.000000000', '2014-03-01T12:00:00.000000000',
       '2014-04-01T12:00:00.000000000', '2014-05-01T12:00:00.000000000',
       '2014-06-01T12:00:00.000000000', '2014-07-01T12:00:00.000000000',
       '2014-08-01T12:00:00.000000000', '2014-09-01T12:00:00.000000000',
       '2014-10-01T12:00:00.000000000', '2014-11-01T12:00:00.000000000',
       '2014-12-01T12:00:00.000000000', '2015-01-01T12:00:00.000000000',
       '2015-02-01T12:00:00.000000000', '2015-03-01T12:00:00.000000000',
       '2015-04-01T12:00:00.000000000', '2015-05-01T12:00:00.000000000',
       '2015-06-01T12:00:00.000000000', '2015-07-01T12:00:00.000000000',
       '2015-08-01T12:00:00.000000000', '2015-09-01T12:00:00.000000000',
       '2015-10-01T12:00:00.000000000', '2015-11-01T12:00:00.000000000',
       '2015-12-01T12:00:00.000000000', '2016-01-01T12:00:00.000000000',
       '2016-02-01T12:00:00.000000000', '2016-03-01T12:00:00.000000000',
       '2016-04-01T12:00:00.000000000', '2016-05-01

In [16]:
# check time bounds
# theta_snaps.time_bnds.values

### Volume

Calculate the volume of each grid cell. This is used when converting advective and diffusive flux convergences and calculating volume-weighted averages.

In [17]:
# Volume (m^3)
# Multiply area, grid cell thickness, and land mask
mask = HH_grid.maskC.where(HH_grid.maskC,np.nan,1)
vol_sassie = ((HH_grid.rAc*HH_grid.drF)*mask).transpose('k','j','i')

In [18]:
# (vol_sassie.isel(k=0)/1e9).plot();

### Open 3D model fields

In [19]:
# open all datasets needed for heat budget
# ADVx_TH_ds = open_s3_mfdataset('ADVx_TH_AVG_DAILY', s3_bucket='s3://ecco-processed-data/SASSIE/N1_test/HH/NETCDF')
# ADVy_TH_ds = open_s3_mfdataset('ADVy_TH_AVG_DAILY', s3_bucket='s3://ecco-processed-data/SASSIE/N1_test/HH/NETCDF')
# ADVr_TH_ds = open_s3_mfdataset('ADVr_TH_AVG_DAILY', s3_bucket='s3://ecco-processed-data/SASSIE/N1_test/HH/NETCDF')
# DFrI_TH_ds = open_s3_mfdataset('DFrI_TH_AVG_DAILY', s3_bucket='s3://ecco-processed-data/SASSIE/N1_test/HH/NETCDF')
# TFLUX_ds = open_s3_mfdataset('TFLUX_AVG_DAILY', s3_bucket='s3://ecco-processed-data/SASSIE/N1_test/HH/NETCDF')
# KPPg_TH_ds = open_s3_mfdataset('KPPg_TH_AVG_DAILY', s3_bucket='s3://ecco-processed-data/SASSIE/N1_test/HH/NETCDF')
# oceQsw_ds = open_s3_mfdataset('oceQsw_AVG_DAILY', s3_bucket='s3://ecco-processed-data/SASSIE/N1_test/HH/NETCDF')
# WTHMASS_ds = open_s3_mfdataset('WTHMASS_AVG_DAILY', s3_bucket='s3://ecco-processed-data/SASSIE/N1_test/HH/NETCDF')

ADVx_TH_ds = open_zarr_store(ADVx_TH_path, s3_options)
ADVy_TH_ds = open_zarr_store(ADVy_TH_path, s3_options)
ADVr_TH_ds = open_zarr_store(ADVr_TH_path, s3_options)
DFrI_TH_ds = open_zarr_store(DFrI_TH_path, s3_options)
TFLUX_ds = open_zarr_store(TFLUX_path, s3_options)
# KPPg_TH_ds = open_zarr_store(KPPg_TH_path, s3_options)
oceQsw_ds = open_zarr_store(oceQsw_path, s3_options)
# WTHMASS_ds = open_zarr_store(WTHMASS_ds, s3_options)

In [20]:
# DAILY mean is labeled at middle of week time period
print(ADVx_TH_ds.isel(time=0).time.values)
print(ADVx_TH_ds.isel(time=0).time_bnds.values)

2014-01-15T12:00:00.000000000
['2014-01-15T00:00:00.000000000' '2014-01-16T00:00:00.000000000']


### Take monthly means

In [21]:
ADVx_TH_ds = ADVx_TH_ds.chunk(chunks={'time':1, 'k':50, 'j':108,'i_g':180}).resample(time='1M').mean()
ADVy_TH_ds = ADVy_TH_ds.chunk(chunks={'time':1, 'k':50, 'j_g':108,'i':180}).resample(time='1M').mean()
ADVr_TH_ds = ADVr_TH_ds.chunk(chunks={'time':1, 'k_l':50, 'j':108,'i':180}).resample(time='1M').mean()
DFrI_TH_ds = DFrI_TH_ds.chunk(chunks={'time':1, 'k_l':50, 'j':108,'i':180}).resample(time='1M').mean()
TFLUX_ds = TFLUX_ds.chunk(chunks={'time':1, 'j':108,'i':180}).resample(time='1M').mean()
oceQsw_ds = oceQsw_ds.chunk(chunks={'time':1, 'j':108,'i':180}).resample(time='1M').mean()

In [22]:
# correct for resampling putting monthly means at the end 

new_time = ADVx_TH_ds.indexes['time'].to_period('M').to_timestamp('M') - pd.offsets.Day(15)

ADVx_TH_ds = ADVx_TH_ds.assign_coords(time=new_time)
ADVy_TH_ds = ADVy_TH_ds.assign_coords(time=new_time)
ADVr_TH_ds = ADVr_TH_ds.assign_coords(time=new_time)
DFrI_TH_ds = DFrI_TH_ds.assign_coords(time=new_time)
TFLUX_ds = TFLUX_ds.assign_coords(time=new_time)
oceQsw_ds = oceQsw_ds.assign_coords(time=new_time)


In [23]:
ADVx_TH_ds = ADVx_TH_ds.isel(time=slice(1,-1))
ADVy_TH_ds = ADVy_TH_ds.isel(time=slice(1,-1))
ADVr_TH_ds = ADVr_TH_ds.isel(time=slice(1,-1))
DFrI_TH_ds = DFrI_TH_ds.isel(time=slice(1,-1))
TFLUX_ds = TFLUX_ds.isel(time=slice(1,-1))
oceQsw_ds = oceQsw_ds.isel(time=slice(1,-1))

In [24]:
print('Number of monthly mean records: ', len(ADVx_TH_ds.time))
print('Number of monthly snapshot records: ', len(theta_snaps.time))

Number of monthly mean records:  84
Number of monthly snapshot records:  85


In [25]:
# ADVx_TH_da = ADVx_TH_ds.isel(time=0).ADVx_TH
# ADVy_TH_da = ADVy_TH_ds.isel(time=0).ADVy_TH
# ADVr_TH_da = ADVr_TH_ds.isel(time=0).ADVr_TH
# DFrI_TH_da = DFrI_TH_ds.isel(time=0).DFrI_TH
# TFLUX_da = TFLUX_ds.isel(time=0).TFLUX
# # KPPg_TH_da = KPPg_TH_ds.isel(time=0).KPPg_TH
# oceQsw_da = oceQsw_ds.isel(time=0).oceQsw
# # WTHMASS_da = WTHMASS_ds.isel(time=0).WTHMASS

In [26]:
# double check dates
print(f"snapshot start and end: {theta_snaps.time.values}")
print(f"\nmonthly mean time: {TFLUX_ds.time.values}")
# print(f"\ndaily mean time start and end: {TFLUX_ds.isel(time=0).time_bnds.values}")

snapshot start and end: ['2014-02-01T12:00:00.000000000' '2014-03-01T12:00:00.000000000'
 '2014-04-01T12:00:00.000000000' '2014-05-01T12:00:00.000000000'
 '2014-06-01T12:00:00.000000000' '2014-07-01T12:00:00.000000000'
 '2014-08-01T12:00:00.000000000' '2014-09-01T12:00:00.000000000'
 '2014-10-01T12:00:00.000000000' '2014-11-01T12:00:00.000000000'
 '2014-12-01T12:00:00.000000000' '2015-01-01T12:00:00.000000000'
 '2015-02-01T12:00:00.000000000' '2015-03-01T12:00:00.000000000'
 '2015-04-01T12:00:00.000000000' '2015-05-01T12:00:00.000000000'
 '2015-06-01T12:00:00.000000000' '2015-07-01T12:00:00.000000000'
 '2015-08-01T12:00:00.000000000' '2015-09-01T12:00:00.000000000'
 '2015-10-01T12:00:00.000000000' '2015-11-01T12:00:00.000000000'
 '2015-12-01T12:00:00.000000000' '2016-01-01T12:00:00.000000000'
 '2016-02-01T12:00:00.000000000' '2016-03-01T12:00:00.000000000'
 '2016-04-01T12:00:00.000000000' '2016-05-01T12:00:00.000000000'
 '2016-06-01T12:00:00.000000000' '2016-07-01T12:00:00.000000000'
 

***

# Calculate total tendency of $\theta$ ($G^{\theta}_\textrm{total}$)

We calculate the daily-averaged time tendency of ``THETA`` by differencing daily ``THETA`` snapshots.

In [27]:
theta_snaps = theta_snaps.chunk(chunks={'time':1, 'k':50, 'j':108,'i':180})
theta_snaps


<xarray.Dataset> Size: 119GB
Dimensions:    (time: 85, k: 90, j: 1080, i: 1800, k_p1: 91, k_l: 90, k_u: 90,
                nv: 2)
Coordinates: (12/14)
    XC         (j, i) float32 8MB dask.array<chunksize=(108, 180), meta=np.ndarray>
    YC         (j, i) float32 8MB dask.array<chunksize=(108, 180), meta=np.ndarray>
    Zp1        (k_p1) float32 364B dask.array<chunksize=(91,), meta=np.ndarray>
  * i          (i) int32 7kB 0 1 2 3 4 5 6 ... 1794 1795 1796 1797 1798 1799
    Zl         (k_l) float32 360B dask.array<chunksize=(90,), meta=np.ndarray>
    Z          (k) float32 360B dask.array<chunksize=(50,), meta=np.ndarray>
    ...         ...
  * k_l        (k_l) int32 360B 0 1 2 3 4 5 6 7 8 ... 81 82 83 84 85 86 87 88 89
  * k_p1       (k_p1) int32 364B 0 1 2 3 4 5 6 7 8 ... 83 84 85 86 87 88 89 90
  * time       (time) datetime64[ns] 680B 2014-02-01T12:00:00 ... 2021-02-01T...
  * j          (j) int32 4kB 0 1 2 3 4 5 6 ... 1074 1075 1076 1077 1078 1079
    time_bnds  (time, nv) datetime64[ns] 1kB dask.array<chunksize=(1, 2), meta=np.ndarray>
  * k_u        (k_u) int32 360B 0 1 2 3 4 5 6 7 8 ... 81 82 83 84 85 86 87 88 89
Dimensions without coordinates: nv
Data variables:
    THETA      (time, k, j, i) float64 119GB dask.array<chunksize=(1, 50, 108, 180), meta=np.ndarray>
Attributes: (12/55)
    Conventions:                     CF-1.8, ACDD-1.3
    acknowledgement:                 This research was carried out by the Jet...
    author:                          Mike Wood, Marie Zahn, and Ian Fenty
    cdm_data_type:                   Grid
    comment:                         SASSIE llc1080 V1R1 fields are consolida...
    creator_email:                   ecco-group@mit.edu
    ...                              ...
    standard_name_vocabulary:        NetCDF Climate and Forecast (CF) Metadat...
    summary:                         This dataset provides data variable and ...
    time_coverage_end:               2021-02-08T00:00:00
    time_coverage_start:             2014-01-15T00:00:00
    title:                           SASSIE Ocean Model THETA Parameter for t...
    uuid:                            92f116d8-13ba-11ef-9cf8-0604868e061f

In [28]:
# Set eta term to zero due to linear free surface
sTHETA = theta_snaps.THETA

In [29]:
# # number of seconds in a day
# secs_per_snap = 86400

dt = theta_snaps.time.diff('time').astype('float32')
dt = dt/1e9

In [30]:
# Total tendency (degC/s)
G_total = sTHETA.diff(dim='time')/np.expand_dims(dt.values,axis=(1,2,3))
G_total.name = 'G_total'

Now convert $\Delta$ `THETA` / $\Delta$ t (day) to $\Delta$ `THETA` / $\Delta$ t (seconds) by dividing by the number of seconds in a day.

In [31]:
# Total tendency
# G_total = G_total_diff / secs_per_snap

### plot surface (k=0)

In [32]:
Z = G_total.isel(k=0).Z.values

In [33]:
theta_snaps.time.isel(time=6).values

np.datetime64('2014-08-01T12:00:00.000000000')

In [34]:
# fig, (ax1) = plt.subplots(1,1,figsize=[10,6])
# G_total.isel(k=0, time=6).plot(ax=ax1,cmap='RdBu_r',vmin=-1e-6,vmax=1e-6);
# Z = G_total.isel(k=0).Z.values
# # ax1.set_title(f"Total temperature tendency on {str(theta_snaps.time.isel(time=6).values[0])[0:10]} from snapshots\nZ={Z} [m], surface (deg C/s)");
# plt.tight_layout()

### plot 55 m

In [35]:
# fig, (ax1) = plt.subplots(1,1,figsize=[10,6])
# G_total.isel(k=16, time=6).plot(ax=ax1,cmap='RdBu_r',vmin=-1e-6,vmax=1e-6);
# Z = G_total.isel(k=0).Z.values
# # ax1.set_title(f"Total temperature tendency on {str(theta_snaps.time.isel(time=6).values[0])[0:10]} from snapshots\nZ={Z} [m], surface (deg C/s)");
# plt.tight_layout()

# Calculate tendency due to advective convergence ($G^{\theta}_\textrm{advection}$)
### Horizontal convergence of advective heat flux
The relevant fields from the diagnostic output here are
- `ADVx_TH`: U Component Advective Flux of Potential Temperature (degC m^3/s)
- `ADVy_TH`: V Component Advective Flux of Potential Temperature (degC m^3/s)

In [36]:
def diff_2d_flux_HH(flux_vector_dict):
    """
    A function (similar to xgcm.Grid.diff_2d_vector) that differences flux variables on the HH Arctic grid.
    """

    u_flux = flux_vector_dict['X']
    v_flux = flux_vector_dict['Y']

    # take differences
    diff_u_flux = u_flux.diff('i_g')
    diff_v_flux = v_flux.diff('j_g')
    
    diff_u_flux_padded = diff_u_flux.pad(pad_width={'i_g':(1,0)},mode='constant',constant_values=np.nan)
    diff_v_flux_padded = diff_v_flux.pad(pad_width={'j_g':(1,0)},mode='constant',constant_values=np.nan)

    # include coordinates of input DataArrays and correct dimension/coordinate names
    diff_u_flux = diff_u_flux_padded.assign_coords(u_flux.coords).rename({'i_g':'i'})
    diff_v_flux = diff_v_flux_padded.assign_coords(v_flux.coords).rename({'j_g':'j'})

    diff_flux_vector_dict = {'X':diff_u_flux,'Y':diff_v_flux}

    return diff_flux_vector_dict

In [37]:
# Set fluxes on land to zero (instead of NaN)
ADVx_TH_ds = ADVx_TH_ds['ADVx_TH'].where(HH_grid.hFacW.values > 0,0)
ADVy_TH_ds = ADVy_TH_ds['ADVy_TH'].where(HH_grid.hFacS.values > 0,0)

# Difference of horizontal components of divergence
ADVxy_diff = diff_2d_flux_HH({'X': ADVx_TH_ds, \
                              'Y': ADVy_TH_ds})

In [38]:
# Convergence of horizontal advection (degC m^3/s)
adv_hConvH_m3 = (-(ADVxy_diff['X'] + ADVxy_diff['Y']))

# divide by volume to get degC/s
adv_hConvH = adv_hConvH_m3 / vol_sassie

### Plot horizontal convergence at the surface

In [39]:
# fig, (ax1) = plt.subplots(1,1,figsize=[10,6])
# adv_hConvH.isel(k=0, time=5).plot(ax=ax1,cmap='RdBu_r',vmin=-1e-5,vmax=1e-5);
# ax1.set_title("Temperature tendency due to horizontal advective flux convergence\nk=0, surface (deg C/s)");
# plt.tight_layout()

### Vertical convergence of advective heat flux
The relevant field from the diagnostic output is
- `ADVr_TH`: Vertical Advective Flux of Potential Temperature (degC m^3/s)

In [40]:
# Pad the bottom
ADVr_TH_padded = ADVr_TH_ds.ADVr_TH.pad(pad_width={'k_l':(0,1)},mode='constant',constant_values=np.nan)

In [41]:
# check padding
# ADVr_TH_padded.isel(k_l=-1,i=0).values

In [42]:
# Take differences to get convergence of vertical advection (degC m^3/s)
ADVz_diff = ADVr_TH_padded.diff('k_l')#.compute()

In [43]:
# change dim name
adv_vert_convergence = ADVz_diff.assign_coords(ADVr_TH_ds.ADVr_TH.coords).rename({'k_l':'k'})

# divide by vol to get degC/s
adv_vert_div = (adv_vert_convergence/vol_sassie)#.compute()

In [44]:
adv_vert_div

<xarray.DataArray (time: 84, k: 90, j: 1080, i: 1800)> Size: 118GB
dask.array<truediv, shape=(84, 90, 1080, 1800), dtype=float64, chunksize=(1, 49, 108, 180), chunktype=numpy.ndarray>
Coordinates:
    XC       (j, i) float32 8MB -128.0 -128.0 -128.0 ... 51.96 51.96 51.96
    YC       (j, i) float32 8MB 48.68 48.73 48.78 48.83 ... 63.41 63.37 63.34
    Zl       (k) float32 360B dask.array<chunksize=(50,), meta=np.ndarray>
  * j        (j) int32 4kB 0 1 2 3 4 5 6 ... 1073 1074 1075 1076 1077 1078 1079
  * i        (i) int32 7kB 0 1 2 3 4 5 6 ... 1793 1794 1795 1796 1797 1798 1799
  * k        (k) int32 360B 0 1 2 3 4 5 6 7 8 9 ... 81 82 83 84 85 86 87 88 89
  * time     (time) datetime64[ns] 672B 2014-02-13 2014-03-16 ... 2021-01-16
    Z        (k) float32 360B -0.5 -1.57 -2.79 ... -6.301e+03 -6.76e+03

### Plot the surface

In [45]:
# fig, (ax1) = plt.subplots(1,1,figsize=[10,6])
# adv_vert_div.isel(k=0, time=6).plot(ax=ax1,cmap='RdBu_r',vmin=-1e-5,vmax=1e-5);
# ax1.set_title("Temperature tendency due to vertical advective flux convergence \nk=0, surface (deg C/s)")
# plt.tight_layout()

### Total convergence of advective flux ($G^{\theta}_\textrm{advection}$)
We can get the total convergence by simply adding the horizontal and vertical component. 

In [46]:
# Sum horizontal and vertical convergences
G_advection = (adv_hConvH + adv_vert_div)#.compute()
G_advection

<xarray.DataArray (time: 84, k: 90, j: 1080, i: 1800)> Size: 118GB
dask.array<add, shape=(84, 90, 1080, 1800), dtype=float64, chunksize=(1, 49, 107, 179), chunktype=numpy.ndarray>
Coordinates:
    Z        (k) float32 360B -0.5 -1.57 -2.79 ... -6.301e+03 -6.76e+03
  * k        (k) int32 360B 0 1 2 3 4 5 6 7 8 9 ... 81 82 83 84 85 86 87 88 89
  * i        (i) int32 7kB 0 1 2 3 4 5 6 ... 1793 1794 1795 1796 1797 1798 1799
  * j        (j) int32 4kB 0 1 2 3 4 5 6 ... 1073 1074 1075 1076 1077 1078 1079
  * time     (time) datetime64[ns] 672B 2014-02-13 2014-03-16 ... 2021-01-16
    XC       (j, i) float32 8MB -128.0 -128.0 -128.0 ... 51.96 51.96 51.96
    YC       (j, i) float32 8MB 48.68 48.73 48.78 48.83 ... 63.41 63.37 63.34
    Zl       (k) float32 360B dask.array<chunksize=(50,), meta=np.ndarray>

In [47]:
# fig, (ax1) = plt.subplots(1,1,figsize=[10,6])
# G_advection.isel(k=0, time=4).plot(ax=ax1,cmap='RdBu_r',vmin=-5e-6,vmax=5e-6);
# ax1.set_title("Temperature tendency due to advective flux convergence\nk=0, surface (deg C/s)")
# plt.tight_layout()

# Calculate tendency due to diffusive convergence ($G^{\theta}_\textrm{diffusion}$)
### Horizontal convergence of diffusive heat flux

**There is no horizontal diffusion for the sassie model**

### Vertical convergence of diffusive heat flux
The relevant fields from the diagnostic output are
- `DFrE_TH`: Vertical Diffusive Flux of Potential Temperature (Explicit part) (degC m^3/s)
- `DFrI_TH`: Vertical Diffusive Flux of Potential Temperature (Implicit part) (degC m^3/s)
> **Note**: Vertical diffusion has both an explicit (`DFrE_TH`) and an implicit (`DFrI_TH`) part.

**There is no explicit diffusion in the sassie model**

In [48]:
# Set fluxes on land to zero (instead of NaN)
DFrI_TH = DFrI_TH_ds.DFrI_TH.where(HH_grid.hFacC.values > 0,0)
DFrI_TH = DFrI_TH.transpose('time','k_l','j','i')

In [49]:
# Pad the bottom
DFrI_TH_padded = DFrI_TH.pad(pad_width={'k_l':(0,1)},mode='constant',constant_values=np.nan)

In [50]:
# check padding
# DFrI_TH_padded.isel(k_l=-1,i=0).values

In [51]:
# Take differences to get convergence of vertical advection (degC m^3/s)
DFrI_TH_diff = DFrI_TH_padded.diff('k_l')#.compute()

In [52]:
# change dim name
difI_vert_convergence = DFrI_TH_diff.assign_coords(DFrI_TH.coords).rename({'k_l':'k'})

# divide by vol to get degC/s
G_diffusion = (difI_vert_convergence / vol_sassie)#.compute()

In [53]:
G_diffusion

<xarray.DataArray (time: 84, k: 90, j: 1080, i: 1800)> Size: 118GB
dask.array<truediv, shape=(84, 90, 1080, 1800), dtype=float64, chunksize=(1, 49, 108, 180), chunktype=numpy.ndarray>
Coordinates:
    XC       (j, i) float32 8MB -128.0 -128.0 -128.0 ... 51.96 51.96 51.96
    Zl       (k) float32 360B dask.array<chunksize=(50,), meta=np.ndarray>
    YC       (j, i) float32 8MB 48.68 48.73 48.78 48.83 ... 63.41 63.37 63.34
  * i        (i) int32 7kB 0 1 2 3 4 5 6 ... 1793 1794 1795 1796 1797 1798 1799
  * k        (k) int32 360B 0 1 2 3 4 5 6 7 8 9 ... 81 82 83 84 85 86 87 88 89
  * j        (j) int32 4kB 0 1 2 3 4 5 6 ... 1073 1074 1075 1076 1077 1078 1079
  * time     (time) datetime64[ns] 672B 2014-02-13 2014-03-16 ... 2021-01-16
    Z        (k) float32 360B -0.5 -1.57 -2.79 ... -6.301e+03 -6.76e+03

### Total convergence of diffusive flux ($G^{\theta}_\textrm{diffusion}$)

This model uses KPP, so we need to add tendency due to KPP non-local term to the above diffusion tendency. In this run there is no KPP term saved. So we will skip this section and estimate KPP as a residual

In [54]:
# # Set fluxes on land to zero (instead of NaN)
# KPPg_TH = KPPg_TH_da.where(HH_grid.hFacC.values > 0,0)
# # KPPg_TH = KPPg_TH.transpose('k_l','j','i')
# KPPg_TH = KPPg_TH.transpose('k','j','i')

In [55]:
# # Pad the bottom
# # KPPg_TH_padded = KPPg_TH.pad(pad_width={'k_l':(0,1)},mode='constant',constant_values=np.nan)
# KPPg_TH_padded = KPPg_TH.pad(pad_width={'k':(0,1)},mode='constant',constant_values=np.nan)

In [56]:
# check padding
# KPPg_TH_padded.isel(k_l=-1,i=0).values

In [57]:
# # Take differences to get convergence of vertical advection (degC m^3/s)
# # KPPg_TH_diff = KPPg_TH_padded.diff('k_l').compute()
# KPPg_TH_diff = KPPg_TH_padded.diff('k').compute()

In [58]:
# # change dim name
# # KPPg_TH_convergence = KPPg_TH_diff.assign_coords(KPPg_TH_da.coords).rename({'k_l':'k'})
# KPPg_TH_convergence = KPPg_TH_diff.assign_coords(KPPg_TH_da.coords).rename({'k':'k'})

# # divide by vol to get degC/s
# KPP_tend = (KPPg_TH_convergence / vol_sassie).compute()

In [59]:
# # for sassie, it is just vertical diffusion
# G_diffusion_KPP = G_diffusion + KPP_tend
# # G_diffusion_KPP = G_diffusion

In [60]:
# fig, (ax1) = plt.subplots(1,1,figsize=[10,6])
# G_diffusion.isel(k=0).plot(ax=ax1,cmap='RdBu_r',vmin=-1e-5,vmax=1e-5);
# ax1.set_title("Temperature tendency due to diffusive flux convergence\nk=0, surface (deg C/s)")
# plt.tight_layout()

In [61]:
# fig, (ax1) = plt.subplots(1,1,figsize=[10,6])
# G_diffusion_KPP.isel(k=0).plot(ax=ax1,cmap='RdBu_r',vmin=-1e-5,vmax=1e-5);
# ax1.set_title("Temperature tendency due to diffusive flux convergence and KPP mixing\nk=0, surface (deg C/s)")
# plt.tight_layout()

# Surface correction
Tendency of mass correction at the surface (due to Linear Free Surface approximation)

`tsurfcor` = 0 because `linFSConserveTr = True` was not specified in the 'data' namelist file in &PARM01. According to the MITgcm defaults listed [here](https://github.com/MITgcm/MITgcm/blob/master/model/src/set_defaults.F), linFSConserveTr = .FALSE. by default.

UNITS check:<br>
WTHMASS is in degC m s-1<br>
degC m s-1 / m = degC s-1

In [62]:
# tsurfcor = 0
# Surf_corr_tend = (tsurfcor - WTHMASS_da) / HH_grid.drF.isel(k=0) * HH_grid.hFacC.isel(k=0)

In [63]:
# fig, (ax1) = plt.subplots(1,1,figsize=[10,6])
# Surf_corr_tend.plot(ax=ax1,cmap='RdBu_r',vmin=-1e-5,vmax=1e-5);
# # Surf_corr_tend.plot(ax=ax1,cmap='RdBu_r',vmin=-1e-12,vmax=1e-12);
# ax1.set_title("Tendency of mass correction at the surface (deg C/s)")
# plt.tight_layout()

# Calculate tendency due to forcing ($G^{\theta}_\textrm{forcing}$)
Finally, we evaluate the local forcing term due to surface heat and geothermal fluxes.

### Surface heat flux
For the surface contribution, there are two relevant model diagnostics:
- `TFLUX`: total heat flux (match heat-content variations) (W/m^2)
- `oceQsw`: net Short-Wave radiation (+=down) (W/m^2)

In [64]:
# Seawater density (kg/m^3)
# needed to convert surface mass fluxes to volume fluxes
rhoconst_sassie = 1027.5

# Heat capacity (J/(kg degC))
c_p_sassie = 3994

# Constants for surface heat penetration (from Table 2 of Paulson and Simpson, 1977)
R = 0.62
zeta1 = 0.6
zeta2 = 20.0

In [65]:
Z = HH_grid.Z.compute()
RF = np.concatenate([HH_grid.Zp1.values[:-1],[np.nan]])

In [66]:
q1 = R*np.exp(1.0/zeta1*RF[:-1]) + (1.0-R)*np.exp(1.0/zeta2*RF[:-1])
q2 = R*np.exp(1.0/zeta1*RF[1:]) + (1.0-R)*np.exp(1.0/zeta2*RF[1:])

In [67]:
# Correction for the 200m cutoff
zCut = np.where(Z < -200)[0][0]
q1[zCut:] = 0
q2[zCut-1:] = 0

In [68]:
# Create xarray data arrays
q1 = xr.DataArray(q1,coords=[Z.k],dims=['k'])
q2 = xr.DataArray(q2,coords=[Z.k],dims=['k'])

In [69]:
## Land masks
# Make copy of hFacC
mskC = HH_grid.hFacC.copy(deep=True).compute()

# Change all fractions (ocean) to 1. land = 0
mskC.values[mskC.values>0] = 1

In [70]:
# Shortwave flux below the surface (W/m^2)
forcH_subsurf = ((q1*(mskC==1)-q2*(mskC.shift(k=-1)==1))*oceQsw_ds.oceQsw).transpose('time','k','j','i')

In [71]:
# Surface heat flux convergence into the uppermost grid cell (W/m^2)
forcH_surf = ((TFLUX_ds.TFLUX - (1-(q1[0]-q2[0]))*oceQsw_ds.oceQsw)\
              *mskC[0]).transpose('time','j','i').assign_coords(k=0).expand_dims('k')

In [72]:
# Full-depth sea surface forcing (W/m^2)
forcH = xr.concat([forcH_surf,forcH_subsurf[:,1:,:,:]], dim='k').transpose('time','k','j','i')
forcH

<xarray.DataArray (time: 84, k: 90, j: 1080, i: 1800)> Size: 118GB
dask.array<transpose, shape=(84, 90, 1080, 1800), dtype=float64, chunksize=(1, 89, 108, 180), chunktype=numpy.ndarray>
Coordinates:
  * i        (i) int32 7kB 0 1 2 3 4 5 6 ... 1793 1794 1795 1796 1797 1798 1799
  * j        (j) int32 4kB 0 1 2 3 4 5 6 ... 1073 1074 1075 1076 1077 1078 1079
    XC       (j, i) float32 8MB -128.0 -128.0 -128.0 ... 51.96 51.96 51.96
    YC       (j, i) float32 8MB 48.68 48.73 48.78 48.83 ... 63.41 63.37 63.34
  * time     (time) datetime64[ns] 672B 2014-02-13 2014-03-16 ... 2021-01-16
  * k        (k) int64 720B 0 1 2 3 4 5 6 7 8 9 ... 81 82 83 84 85 86 87 88 89
    Z        (k) float32 360B -0.5 -1.57 -2.79 ... -6.301e+03 -6.76e+03

***

#### Alternate approach to calculating forcing that directly reflects [MITgcm reference](http://mitgcm.org/download/daily_snapshot/MITgcm/doc/Heat_Salt_Budget_MITgcm.pdf).

Tendency due to shortwave penetration:

In [73]:
# depth = HH_grid.Zp1.values[:-1] # equivalent to RF
# depth1 = HH_grid.Zp1.values[1:]

In [74]:
# # check values
# print(depth[0])
# print(depth1[0])

In [75]:
# swfrac = 0.62*np.exp(depth/0.6) + (1.0-0.62)*np.exp(depth/20)
# swfrac1 = 0.62*np.exp(depth1/0.6) + (1.0-0.62)*np.exp(depth1/20.0)

In [76]:
# # 3D field
# Qsw_tend = oceQsw_da/(rhoconst_sassie*c_p_sassie)/(HH_grid.drF*HH_grid.hFacC)*(swfrac-swfrac1)

Tendency at surface due to incident heat flux:

In [77]:
# # 2D field for the surface
# Tflx_tend = (TFLUX_da-oceQsw_da)/(rhoconst_sassie*c_p_sassie*HH_grid.drF.isel(k=0)*HH_grid.hFacC.isel(k=0))

***

### Total forcing ($G^{\theta}_\textrm{forcing}$)

In [78]:
# convert from W/m^2 to degC/s
G_forcing = ((forcH/(rhoconst_sassie*c_p_sassie))/(HH_grid.hFacC*HH_grid.drF))#.compute()

In [79]:
# fig, (ax1) = plt.subplots(1,1,figsize=[10,6])
# G_forcing.isel(k=0, time=0).plot(ax=ax1,cmap='RdBu_r',vmin=-1e-5,vmax=1e-5);
# # ax1.set_title(f"Temperature tendency due to surface heat and geothermal forcing\n{str(theta_snaps.time.values[0])[0:10]}\nk=0, surface (deg C/s)")
# plt.tight_layout()

# Budget closure

***

## First use approach outlined in MITgcm documentation that separates the surface and subsurface

#### Just the surface layer

In [80]:
print(G_advection.isel(k=0).shape)
print(G_diffusion.isel(k=0).shape)
# print(KPP_tend.isel(k=0).shape)
# print(Surf_corr_tend.shape)

# print(Tflx_tend.shape)
# print(Qsw_tend.isel(k=0).shape)

(84, 1080, 1800)
(84, 1080, 1800)


In [81]:
# surf_heat_tend = G_advection.isel(k=0) + G_diffusion.isel(k=0) + Qsw_tend.isel(k=0) + Tflx_tend + Surf_corr_tend + KPP_tend.isel(k=0)

In [82]:
# res_surface = surf_heat_tend-G_total.isel(k=0)

In [83]:
# res_surface.plot(cmap='RdBu_r',vmin=-1e-5,vmax=1e-5,figsize=[9,5]);

#### Below the surface layer

In [84]:
# KPP_tend.isel(k=slice(1,90)).shape

In [85]:
# sub_surf_heat_tend = G_advection.isel(k=slice(1,90)) + G_diffusion.isel(k=slice(1,90)) + Qsw_tend.isel(k=slice(1,90)) + KPP_tend.isel(k=slice(1,90))

In [86]:
# res_sub_surface = sub_surf_heat_tend-G_total.isel(k=slice(1,90)).values

In [87]:
# res_sub_surface.isel(k=3).Z.values

In [88]:
# res_sub_surface.isel(k=3).plot(cmap='RdBu_r',vmin=-1e-5,vmax=1e-5,figsize=[9,5]);

In [89]:
# res_sub_surface.isel(k=29).Z.values

In [90]:
# res_sub_surface.isel(k=29).plot(cmap='RdBu_r',vmin=-1e-5,vmax=1e-5,figsize=[9,5]);

***

## Now use approach in ECCO documentation

In [91]:
# double check dims
print(G_advection.shape)
print(G_diffusion.shape)
# print(KPP_tend.shape)
print(G_forcing.shape)
# print(Surf_corr_tend.shape)

(84, 90, 1080, 1800)
(84, 90, 1080, 1800)
(84, 90, 1080, 1800)


In [92]:
# Sum of terms in RHS of equation
# rhs_sassie = G_advection + G_diffusion + G_forcing #+ KPP_tend
rhs_sassie_noKPP = G_advection + G_diffusion + G_forcing

In [93]:
rhs_sassie_noKPP.shape

(84, 90, 1080, 1800)

Next add the surface correction to k=0

### RHS without surface correction

In [94]:
# fig, (ax1) = plt.subplots(1,1,figsize=[10,6])
# rhs_sassie_noKPP.isel(k=0, time=0).plot(ax=ax1,cmap='RdBu_r',vmin=-1e-5,vmax=1e-5);
# ax1.set_title("RHS (G_advection + G_diffusion + KPP + G_forcing)\nk=0, surface (deg C/s)")
# plt.tight_layout()

In [95]:
G_total = G_total.isel(time=slice(0,83))

In [96]:
G_advection = G_advection.isel(time=slice(1,84))

In [97]:
G_diffusion = G_diffusion.isel(time=slice(1,84))

In [98]:
G_forcing = G_forcing.isel(time=slice(1,84))

In [99]:
G_forcing+G_diffusion+G_advection

<xarray.DataArray (time: 83, k: 90, j: 1080, i: 1800)> Size: 116GB
dask.array<add, shape=(83, 90, 1080, 1800), dtype=float64, chunksize=(1, 48, 107, 179), chunktype=numpy.ndarray>
Coordinates:
  * i        (i) int32 7kB 0 1 2 3 4 5 6 ... 1793 1794 1795 1796 1797 1798 1799
  * j        (j) int32 4kB 0 1 2 3 4 5 6 ... 1073 1074 1075 1076 1077 1078 1079
    XC       (j, i) float32 8MB -128.0 -128.0 -128.0 ... 51.96 51.96 51.96
    YC       (j, i) float32 8MB 48.68 48.73 48.78 48.83 ... 63.41 63.37 63.34
  * time     (time) datetime64[ns] 664B 2014-03-16 2014-04-15 ... 2021-01-16
  * k        (k) int64 720B 0 1 2 3 4 5 6 7 8 9 ... 81 82 83 84 85 86 87 88 89
    Z        (k) float32 360B -0.5 -1.57 -2.79 ... -6.301e+03 -6.76e+03
    Zl       (k) float32 360B 0.0 -1.0 -2.14 ... -5.682e+03 -6.082e+03 -6.52e+03

In [100]:
G_total['time'] = G_forcing['time']

In [101]:
res_KPP = G_total.isel(k=slice(1,50)) - \
            (G_advection.isel(k=slice(1,50)) +\
            G_diffusion.isel(k=slice(1,50)) + \
            G_forcing.isel(k=slice(1,50)))

In [102]:
res_KPP

<xarray.DataArray (time: 83, k: 49, j: 1080, i: 1800)> Size: 63GB
dask.array<sub, shape=(83, 49, 1080, 1800), dtype=float64, chunksize=(1, 48, 107, 179), chunktype=numpy.ndarray>
Coordinates:
    XC       (j, i) float32 8MB -128.0 -128.0 -128.0 ... 51.96 51.96 51.96
    YC       (j, i) float32 8MB 48.68 48.73 48.78 48.83 ... 63.41 63.37 63.34
  * i        (i) int32 7kB 0 1 2 3 4 5 6 ... 1793 1794 1795 1796 1797 1798 1799
    Z        (k) float32 196B -1.57 -2.79 -4.185 -5.78 ... -812.7 -855.8 -900.1
  * k        (k) int32 196B 1 2 3 4 5 6 7 8 9 10 ... 41 42 43 44 45 46 47 48 49
  * j        (j) int32 4kB 0 1 2 3 4 5 6 ... 1073 1074 1075 1076 1077 1078 1079
  * time     (time) datetime64[ns] 664B 2014-03-16 2014-04-15 ... 2021-01-16
    Zl       (k) float32 196B -1.0 -2.14 -3.44 -4.93 ... -791.3 -834.0 -877.7

In [103]:
k0 = res_KPP.isel(k=0).copy(deep=True)  # Copy shape & metadata
k0[:] = np.nan  # or 0, or some other value

# Set the new k value
k0 = k0.assign_coords(k=0)

# Concatenate k=0 layer at the beginning
res_KPP_new = xr.concat([k0, res_KPP], dim='k')

In [104]:
res_KPP_new = res_KPP_new.transpose('time','k','j','i')

In [105]:
res_KPP_new

<xarray.DataArray (time: 83, k: 50, j: 1080, i: 1800)> Size: 65GB
dask.array<transpose, shape=(83, 50, 1080, 1800), dtype=float64, chunksize=(1, 48, 107, 179), chunktype=numpy.ndarray>
Coordinates:
    XC       (j, i) float32 8MB -128.0 -128.0 -128.0 ... 51.96 51.96 51.96
    YC       (j, i) float32 8MB 48.68 48.73 48.78 48.83 ... 63.41 63.37 63.34
  * i        (i) int32 7kB 0 1 2 3 4 5 6 ... 1793 1794 1795 1796 1797 1798 1799
    Z        (k) float32 200B -1.57 -1.57 -2.79 -4.185 ... -812.7 -855.8 -900.1
  * k        (k) int64 400B 0 1 2 3 4 5 6 7 8 9 ... 41 42 43 44 45 46 47 48 49
  * j        (j) int32 4kB 0 1 2 3 4 5 6 ... 1073 1074 1075 1076 1077 1078 1079
  * time     (time) datetime64[ns] 664B 2014-03-16 2014-04-15 ... 2021-01-16
    Zl       (k) float32 200B -1.0 -1.0 -2.14 -3.44 ... -791.3 -834.0 -877.7

In [106]:
surf_corr = G_total - (G_advection + G_diffusion +  G_forcing + res_KPP_new)

In [107]:
##This code did not write the advection and diffusion term!!!!! I had to modify it to save them
### OKay, I'm gonna have to save the terms in separate files
varnames = ['G_forcing']

# G_total = G_total.transpose('time','tile','k','j','i')

# adv_diff_written = True

ds_budg = xr.Dataset(data_vars={})
for varname in varnames:
    # if varname not in globals():
        # create empty dask arrays for G_advection and G_diffusion (to be written later)
        # import dask.array as da
        # ds_budg[varname] = (['time','tile','k','j','i'],\
        #                     da.empty((ds.sizes['time'],13,50,90,90),dtype='float32',\
        #                             chunks=(1,13,50,90,90)))
        # # adv_diff_written = False
    # else:
    ds_budg[varname] = globals()[varname].chunk(chunks={'time':1,'k':50,'j':108,'i':180})        

In [108]:
# # Add surface forcing (degC/s)
# ds_budg['Qnet'] = ((forcH /(rhoconst*c_p))\
#                   /(ecco_grid.hFacC*ecco_grid.drF)).chunk(chunks={'time':1,'tile':13,'k':50,'j':90,'i':90})

In [109]:
# # Add shortwave penetrative flux (degC/s)
# #Since we only are interested in the subsurface heat flux we need to zero out the top cell
# SWpen = ((forcH_subsurf /(rhoconst*c_p))/(ecco_grid.hFacC*ecco_grid.drF)).where(forcH_subsurf.k>0).fillna(0.)
# ds_budg['SWpen'] = SWpen.where(ecco_grid.hFacC>0).chunk(chunks={'time':1,'tile':13,'k':50,'j':90,'i':90})

In [110]:
ds_budg.time.encoding = {}
ds_budg = ds_budg.reset_coords(drop=True)

In [111]:
ds_budg

<xarray.Dataset> Size: 116GB
Dimensions:    (i: 1800, j: 1080, time: 83, k: 90)
Coordinates:
  * i          (i) int32 7kB 0 1 2 3 4 5 6 ... 1794 1795 1796 1797 1798 1799
  * j          (j) int32 4kB 0 1 2 3 4 5 6 ... 1074 1075 1076 1077 1078 1079
  * time       (time) datetime64[ns] 664B 2014-03-16 2014-04-15 ... 2021-01-16
  * k          (k) int64 720B 0 1 2 3 4 5 6 7 8 9 ... 81 82 83 84 85 86 87 88 89
Data variables:
    G_forcing  (time, k, j, i) float64 116GB dask.array<chunksize=(1, 50, 108, 180), meta=np.ndarray>

In [112]:
import psutil
from os.path import join,expanduser
import sys
user_home_dir = expanduser('~')


# save_dir is set to ~/Downloads below;
# change if you want to save somewhere else
save_dir = join(user_home_dir,'efs_ecco/asiddiqu/llc1080_heat_budget_terms/')

# first query how much storage is free
# the zarr file will occupy ~15 GB, so require 20 GB free storage as a buffer

import shutil
free_storage = shutil.disk_usage(save_dir).free
print(f'Free storage: {free_storage/(10**9)} GB')

# query how much memory is available
# (influences how this large archive will be computed and stored)
mem_avail = psutil.virtual_memory().available
print('Available memory:',mem_avail/(10**9),'GB')

Free storage: 9223359965.920494 GB
Available memory: 209.084227584 GB


In [113]:
from dask.diagnostics import ProgressBar

In [114]:
# for var in ds_budg.data_vars:
#     if ds_budg[var].dtype == 'float64':
#         ds_budg[var] = ds_budg[var].astype('float32')

# ds_budg

In [115]:
# # to save budget as netcdf, set save_netcdf = True
# save_netcdf = True

# # encoding = {var: {"dtype": "float32", "zlib": True, "complevel": 1} for var in ds_budg.data_vars}

# # the netcdf file will occupy ~53 GB, so require 60 GB free storage as a buffer
# if save_netcdf:
#     if free_storage >= 60*(10**9):
#         with ProgressBar():
#             ds_budg.to_netcdf(join(save_dir,'llc1080_budg_heat_frc.nc'), compute=True)
#     else:
#         print('Insufficient storage to save global budget terms to disk as netcdf')

In [116]:
save_dir = "/home/jovyan/efs_ecco/asiddiqu/llc1080_heat_budget_terms"  # ✅ Replace with your actual path
chunk_dir = join(save_dir, "llc1080_frc_chunks")
os.makedirs(chunk_dir, exist_ok=True)

In [117]:
# %%time
# with ProgressBar():
#     for i in range(ds_budg.dims['time']):
#         subset = ds_budg.isel(time=i, drop=True)
#         filename = f"llc1080_budg_time_{i:03d}.nc"
#         filepath = join(chunk_dir, filename)
#         subset.to_netcdf(filepath, compute=True)

In [ ]:
%%time
varnames = ['G_advection']


ds_budg = xr.Dataset(data_vars={})
for varname in varnames:
    ds_budg[varname] = globals()[varname].chunk(chunks={'time':1,'k':50,'j':108,'i':180}) 

ds_budg.time.encoding = {}
ds_budg = ds_budg.reset_coords(drop=True)

chunk_dir = join(save_dir, "llc1080_adv_chunks")
os.makedirs(chunk_dir, exist_ok=True)

with ProgressBar():
    for i in range(ds_budg.dims['time']):
        subset = ds_budg.isel(time=i, drop=True)
        filename = f"llc1080_budg_time_{i:03d}.nc"
        filepath = join(chunk_dir, filename)
        subset.to_netcdf(filepath, compute=True)

In [119]:
%%time
varnames = ['G_diffusion']


ds_budg = xr.Dataset(data_vars={})
for varname in varnames:
    ds_budg[varname] = globals()[varname].chunk(chunks={'time':1,'k':50,'j':108,'i':180}) 

ds_budg.time.encoding = {}
ds_budg = ds_budg.reset_coords(drop=True)

chunk_dir = join(save_dir, "llc1080_dif_chunks")
os.makedirs(chunk_dir, exist_ok=True)

with ProgressBar():
    for i in range(ds_budg.dims['time']):
        subset = ds_budg.isel(time=i, drop=True)
        filename = f"llc1080_budg_time_{i:03d}.nc"
        filepath = join(chunk_dir, filename)
        subset.to_netcdf(filepath, compute=True)

UsageError: Line magic function `%%time` not found.


In [ ]:
%%time
varnames = ['G_total']


ds_budg = xr.Dataset(data_vars={})
for varname in varnames:
    ds_budg[varname] = globals()[varname].chunk(chunks={'time':1,'k':50,'j':108,'i':180}) 

ds_budg.time.encoding = {}
ds_budg = ds_budg.reset_coords(drop=True)

chunk_dir = join(save_dir, "llc1080_tnd_chunks")
os.makedirs(chunk_dir, exist_ok=True)

%%time
with ProgressBar():
    for i in range(ds_budg.dims['time']):
        subset = ds_budg.isel(time=i, drop=True)
        filename = f"llc1080_budg_time_{i:03d}.nc"
        filepath = join(chunk_dir, filename)
        subset.to_netcdf(filepath, compute=True)